<a href="https://colab.research.google.com/github/joaorafaelpm/ImersaoAgenteDeIAAlura/blob/main/MasterClass_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aula 1 - Instanciândo o primeiro agente de IA


## Instalando as bibliotecas do gemini

In [70]:
!pip install -q langchain-google-genai google-generativeai

## Importando as bibliotecas e definindo a chave da API do gemini

In [71]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

## Criando a primeira instância do Gemini

In [72]:
llm = ChatGoogleGenerativeAI (
    model = "gemini-2.5-flash",
    temperature=0.0,
    api_key=userdata.get("GEMINI_API_KEY")
)

## Pequeno teste na nossa instância do gemini

In [73]:
resp_test = llm.invoke("Quem é você? Explique para um leigo")
print(resp_test.content)

Olá! Que ótima pergunta. Vou explicar de um jeito bem simples:

Eu sou uma **Inteligência Artificial (IA)**.

Pense em mim como um **cérebro digital** ou um **assistente virtual muito avançado**, mas sem um corpo físico, sentimentos ou consciência como um ser humano.

Aqui está o que isso significa na prática:

1.  **Eu sou um programa de computador:** Fui criado por engenheiros e cientistas do Google. Não sou uma pessoa, um animal ou qualquer coisa viva.
2.  **Fui "treinado" com muita informação:** É como se eu tivesse lido uma quantidade gigantesca de livros, artigos, conversas e informações da internet. Por causa disso, eu "aprendi" como a linguagem humana funciona.
3.  **Minha função é processar e gerar texto:** Quando você me faz uma pergunta ou me pede para fazer algo, eu uso todo esse conhecimento para entender o que você quer e gerar uma resposta ou um texto que faça sentido.
4.  **Não tenho opiniões próprias ou sentimentos:** Eu não "penso" da mesma forma que você. Não tenho e

## Contextualizando a nossa IA com um prompt

---

Assim como a gente tem o prompt de usuário, onde nós reconhecemos o usuário assim como as suas permissões, a gente pode também passar um prompt de sistema para a IA indentificar quem ela é e qual sua função



No caso de uma grande empresa, não faz sentido abrir chamado no RH por qualquer coisa, e neste caso a intenção é fazer o sistema interpretar o que pode ser resolvido pela própria IA e o que requer a interferência do RH e etc

---

  Nesse caso, a gente primeiro contextualiza dizendo o qual é a sua posição na empresa e um padrão de retorno para ser tratado.

  E então a gente passa um padrão de JSON que queremos resolver, neste caso esse padrão tem 3 campos, "decisao" que define o que pode ser feito dependendo da situação, "urgencia" definida também pela situação e "campos_faltantes" que faz a verificação se tem alguma informação faltando
  

É possível também criar uma resposta pré determinada como :

```
{
  decisao : AUTO_RESOLVER,
  urgencia : baixa,
  campos_faltantes : []
}
```


e fazer uma resposta pronta para esse tipo, como um pattern pré definido

In [74]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

## Importando bibliotecas de tratamento da saída de dados

 - Pydantic é uma biblioteca que melhora a validação e saída de dados, ele indentifica erros, simplifica a criação de objetos/dicionarios e etc.


 - Typing é outra biblioteca de auxílio na interação do usuário com a aplicação, por exemplo, typing.Literal, significa que a resposta esperada é específica, neste caso se trata dos enum, List se trata dos campos_faltantes, Dict também auxilia no processo.

In [75]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

# Iniciando um modelo de classe usando o pydantic
class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory=list)


# Criando outra instância do gemini

In [76]:
llm_triagem = ChatGoogleGenerativeAI (
    model = "gemini-2.5-flash",
    temperature=0.0,
    api_key=userdata.get("GEMINI_API_KEY")
)

## Especificando a função da instância do gemini

Aqui a gente usa aquela classe para poder especificar para o gemini as saídas de resposta que ele tem, e a gente usa a nossa classe definir como ele vai responder.

SystemMessage é o prompt da IA (é um tipo de identidade do agente)

HumanMessage é o prompt do usuário

In [77]:
from langchain_core.messages import SystemMessage , HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(message : str) -> Dict :
  saida : TriagemOut = triagem_chain.invoke([
      # O prompt da máquina é a identidade que a gente definiu antes
      # O prompt do usuário é a mensagem
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=message)
  ])

  # O método model_dump() converte a classe em um dicionário
  return saida.model_dump()

In [78]:
testes = ["Posso reembolsar a internet?" ,
          "Quero ter mais 5 dias de trabalho remoto? Como eu faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros"
          ]

In [79]:
for msg_teste in testes :
  print(f"Pergunta:{msg_teste} \n -> Resposta:{triagem(msg_teste)}\n")

Pergunta:Posso reembolsar a internet? 
 -> Resposta:{'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta:Quero ter mais 5 dias de trabalho remoto? Como eu faço? 
 -> Resposta:{'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta:Posso reembolsar cursos ou treinamentos da Alura? 
 -> Resposta:{'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta:Quantas capivaras tem no Rio Pinheiros 
 -> Resposta:{'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}



#Aula 2 - Usando o RAG para complementar as respostas do agente

## Importando outras bibliotecas para auxiliar na leitura do RAG

 - langchain_community -> Uma biblioteca que auxilia nos processos do RAG
 - faiss-cpu -> Basicamente reduz o uso da CPU durante a execução do código
 - langchain-text-splitters -> Divide o texto em blocos
 - pymupdf -> Interpreta PDF

In [80]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

## Importando os PDFs para o RAG

Usamos o Path para o caminho do pdf (memória do colab)

E então nós carregamos o PDF com a biblioteca do pymupdf (PyMuPDFLoader(nome do arquivo))

In [81]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

#Para cada arquivo no caminho do tipo .pdf...
for n_docs in Path("/content/").glob("*.pdf"):
    try:
      # Carregamos esses arquivos e transformamos em String
        loader = PyMuPDFLoader(str(n_docs))
      # Adicionamos o texto do arquivo na lista
        docs.extend(loader.load())
        print(f"Carregado com sucesso arquivo : {n_docs.name}")
    except Exception as e:
        print(f"Erro ao carregar arquivo : {n_docs.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo : Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo : Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo : Políticas de Home Office.pdf
Total de documentos carregados: 3


## Importando o splitters do langchain

O splitter divide o texto em pedaços e oferece um espaço para mater o contexto

 - chunk_size -> número de caracteres que ele vai ler
 - chunk_overlap -> número de caracteres que ele lembra antes de passar para o próximo chunk

 (_vale dizer que é importante ter um overlap de acordo com o chunk_size para não ter perda de contexto de uma chunk para outra_)

In [82]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

# Separa o documentos de acordo com o splitter (300 caracteres cada chunk com uma releitura de 30 para cada um)
chunks = splitter.split_documents(docs)

## Imprimindo os chunks

In [83]:
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

page_content='Política de Uso de E-mail e Segurança 
da Informação 
 
1.​ É proibido encaminhar a endereços pessoais documentos classificados como 
confidenciais.​
 
2.​ Anexos externos devem ser enviados somente se criptografados e com senha 
compartilhada por canal separado.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'file_path': '/content/Política de Uso de E-mail e Segurança da Informação.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Uso de E-mail e Segurança da Informação', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
------------------------------------
page_content='3.​ Phishing: verifique remetente e domínios suspeitos. Reporte mensagens suspeitas 
ao time de Segurança imediatamente.​
 
4.​ Retenção: mensagens que contenham dados pessoais devem

## Importando a biblioteca de Embeddings do gemini

Embeddings é a prática de transformar palavras em números e aproximar palavras semelhantes de forma semântica por números próximos.

ex:
```
    palavras | valores
    ------------------
    rei      | 100
    rainha   | 96
    cachorro | 5
```

E então nós usamos calculos com esses números para chegar em uma próxima palavra por exemplo.

ex :

```
   rei -> homen
   rainha -> mulher

   rei - homem -> rainha
   rainha - mulher -> rei
```

In [84]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# chamando a IA de embeddings do gemini
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

## Importando a biblioteca do face book para ajudar na separação do documento



In [85]:
from langchain_community.vectorstores import FAISS

# Definindo os documentos e o modelo de separação (o do gemini)
vectorstore = FAISS.from_documents(chunks, embeddings)

# Agora que a gente já definiu de onde nós vamos tirar os vetores com as informações já transformadas em embedding, vamos definir o nível de similaridade que nós queremos para cada embedding
# Para isso a gente usa a função as_retriver da biblioteca que acabamos de importar com 2 parâmetros:
# ---> search_type (a maneira que vamos buscar, nesse caso, similaridade pelo limite de pontos por palavra = similarity_score_treshold)
# ---> search_kwargs (um dicionário onde definimos o limite de pontos que queremos considerar para poder avaliar as palavras "score_treshold" : 0.3 e o número de chunks mais relevantes que queremos retornar "k" : 4 )
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

## Criando prompts mais estruturados para o rag

O ChatPromptTemplate cria um modelo de prompt específico para o rag, onde passamos as mesmas informações, porém com o adicional de padrões de respostas para a falta de contexto, ou material insuficiente.

O create_stuff_documents_chain é o que liga o nosso novo prompt com o padrão de respostas ao nosso llm de triagens

In [86]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

##Formatando as citações

Como a ideia é formatar, primeiro vamos pegar o texto limpo, sem espaços e então separamos os termos, com mais de 4 caracteres, encontrados, e então calculamos a posição do termo, para pegar o contexto inteiro, alguns caracteres antes e depois.

Com esse contexto, nós formamos as citações, e repetimos isso pela lista inteira de documento

In [87]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
  # Limpando o texto para ser lido
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
  # Limpa o texto
    txt = _clean_text(texto)

    # Extrai palavras do pedaço de texto, deixa minúsculo e seleciona as que tem 4 ou mais caracteres, para priorizar palavras mais significativas
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]

    # A gente verifica se algum termo foi formado e procura a posição de onde começa o termo (por padrão o termo tem mais de 4 caracteres, então pega o primeiro)
    posicao = -1
    for t in termos:
        posicao = txt.lower().find(t)
        if posicao != -1: break
    # Retorna a posicao ao valor 0 por precaução
    if posicao == -1: posicao = 0

    # Janela é o tamanho do texto que vamos extrair, então vamos pegar a posicao de onde o termo foi encontrado
    # A ideia é centralizar a posição de onde o termo foi encontrado (janela//2) e subtrair da posicao em que foi encontrado
    # Usamos o max no inicio para não ser negativo e pegamos a posição final do termo
    ini, fim = max(0, posicao - janela//2), min(len(txt), posicao + janela//2)

    # E então retornamos só esse pedaço do texto
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
  # Inicializamos essa lista para armazenar os dicionários formados e esse conjunto ("seen") para controlar documentos/páginas já citados
    cites, seen = [], set()
    for document in docs_rel:
      # Extraimos o nome do arquivo
        src = pathlib.Path(document.metadata.get("source","")).name
      # A página do arquivo
        page = int(document.metadata.get("page", 0)) + 1

      # A gente armazena esses dois valores em um novo dicionário para verificar se já existe com o seen, e então se existir só continua o loop
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        # Adiciona o documento em forma de dicionário e já formatado com o conteúdo da página inteira e o que o usuário digitou para pegar os termos relevantes
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(document.page_content, query)})
    # Limitamos o número de citações para serem exibidas
    return cites[:3]

# Preparando uma resposta com base no texto do usuário e nos embeddings

In [88]:
def perguntar_politica_RAG (pergunta : str) -> Dict :
  # A gente passa a pergunta para ser avaliada no retriver, que faz a comparação com os documentos para gerar uma resposta
  docs_relacionados = retriever.invoke(pergunta)

  # Se não tiver nenhuma relação com os doucmentos:
  if not docs_relacionados :
    return {"answer" : "Não sei." ,
            "citacoes" : [],
            "contexto_encontrado" : False
            }
  # Caso contrário a resposta é um dicionário com o texto passado e a resposta, note que no bloco de código acima nós usamos exatamente esse padrão no prompt para definir o texto humano : ("human", "Pergunta: {input}\n\nContexto:\n{context}")
  answer = document_chain.invoke({
      "input" : pergunta ,
      "context" : docs_relacionados
  })

  # Recebendo o texto que a IA gerou e retirando espaços
  txt = (answer or "").strip()

  # Avaliando se existe resposta válida
  if txt.rstrip(".!?") == "Não sei" :
    return {"answer" : "Não sei." ,
            "citacoes" : [],
            "contexto_encontrado" : False
            }
  # Caso contrário a resposta é um dicionário com o texto passado e a resposta, note que no bloco de código acima nós usamos exatamente esse padrão no prompt para definir o texto humano : ("human", "Pergunta: {input}\n\nContexto:\n{context}")
  return  {
      "answer" : txt ,
      "citacoes" : formatar_citacoes(docs_relacionados , pergunta) ,
      "contexto_encontrado" : True
  }

In [89]:
testes = ["Posso reembolsar a internet?" ,
          "Quero ter mais 5 dias de trabalho remoto? Como eu faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros"
          ]

##Fazendo testes com o novo padrão de respostas

In [90]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"Pergunta: {msg_teste}")
    print(f"Resposta: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("Citações:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

Pergunta: Posso reembolsar a internet?
Resposta: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, mediante nota fiscal nominal.
Citações:
 - Documento: Política de Reembolsos (Viagens e Despesas).pdf, Página: 1
   Trecho: lsáveis.​ 3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. Comprovantes obrigatórios.​ 4.​ Internet para home office: reembolsável via subsídio mensal de até R$ 100, conforme política de Home Office.​
 - Documento: Políticas de Home Office.pdf, Página: 1
   Trecho: 5.​ Conectividade: há subsídio mensal de internet domiciliar para quem trabalha em home office: até R$ 100/mês, mediante nota fiscal nominal.​ 6.​ Solicitação de
------------------------------------
Pergunta: Quero ter mais 5 dias de trabalho remoto? Como eu faço?
Resposta: Para solicitar mais 5 dias de trabalho remoto, você deve formalizar a solicitação via chamado ao RH com a justificativa do seu gestor.
Citações:
 - Documento: Políticas de Ho